## 0. 라이브러리 다운로드

### 라이브러리 설명

- 오디오 처리: numpy, librosa, soundfile, pydub
- 모델 실행: torch, transformers, accelerate
- 응용 프로그램 구성: langchain, sentence-transformers
- 데이터셋: datasets

## 설치한 라이브러리들 설명  

### 핵심 AI 및 NLP 관련 라이브러리
1. `torch`  
PyTorch는 딥러닝 모델을 구축하고 훈련하는 데 사용하는 오픈소스 라이브러리입니다.  
GPU 가속을 지원하며, 특히 자연어 처리와 컴퓨터 비전에서 널리 사용됩니다.  
Transformers 모델을 실행하거나 커스텀 모델을 훈련할 때 사용됩니다.  

2. `transformers`  
Hugging Face에서 제공하는 라이브러리로, 사전 학습된 NLP 모델(BERT, GPT, Whisper 등)을 쉽게 사용할 수 있습니다.  
Whisper 모델도 이 라이브러리를 통해 로드하여 사용합니다.  
Whisper 모델로 음성 데이터를 텍스트로 변환하는 데 사용됩니다.  

3. `accelerate`  
Hugging Face에서 제공하는 라이브러리로, 모델 훈련과 추론을 가속화하고 여러 디바이스(CPU, GPU, TPU 등)를 활용하도록 지원합니다.  
Whisper나 기타 Transformer 기반 모델을 실행 시 성능 최적화에 사용됩니다.  

4. `langchain`  
대규모 언어 모델(LLM)을 활용한 애플리케이션을 구축하는 프레임워크입니다.  
여러 NLP 작업(질의응답, 대화 생성 등)을 연결하는 워크플로우를 구성할 때 사용됩니다.  
LLM 기반 애플리케이션 개발에 사용됩니다. Whisper와 결합하여 음성 인식 후 처리 로직을 작성할 때 유용할 수 있습니다.  

1. `sentence-transformers`  
문장 수준의 임베딩(벡터 표현)을 생성하는 데 사용되는 라이브러리입니다.  
텍스트 데이터의 유사도 측정이나 검색 작업에 널리 사용됩니다.  
Whisper로 변환된 텍스트 데이터를 처리하거나 분석하는 데 사용될 수 있습니다.  


### 오디오 처리 및 데이터 관련 라이브러리  
1. `numpy==1.23.4`  
Python의 대표적인 수치 계산 라이브러리로, 행렬 연산 및 고성능 배열 처리를 지원합니다.  
Whisper 모델 및 오디오 데이터 처리에서 핵심적인 역할을 합니다.  
PCM 데이터를 처리하거나 librosa와 함께 오디오 데이터 배열을 조작할 때 사용됩니다.  
(`librosa`와 호환하기 위해 버전을 지정해주었습니다. 2.0.0 버전도 가능)  

2. `librosa`  
오디오 분석과 신호 처리를 위한 라이브러리입니다.  
오디오 데이터를 주파수 영역(Mel Spectrogram)으로 변환하는 등 Whisper 모델과 직접적으로 연관이 있습니다.  
Whisper 모델에 입력으로 제공되는 데이터를 전처리하거나 변환하는 데 사용됩니다.  

3. `soundfile`  
오디오 파일을 읽고 쓰는 데 사용되는 라이브러리입니다.  
`librosa`가 내부적으로 의존합니다.  
오디오 데이터를 로드하거나 저장할 때 사용됩니다.  

4. `pydub`  
오디오 데이터를 자르거나 합치는 등의 작업을 지원하는 라이브러리입니다.  
ffmpeg와 함께 동작하며, Whisper에 입력으로 제공할 오디오를 준비하는 데 유용합니다.  
긴 오디오 데이터를 30초 단위로 분할하거나 특정 포맷으로 변환할 때 사용됩니다.  

In [1]:
# !pip install torch transformers accelerate langchain sentence-transformers

In [2]:
# !pip install numpy==1.23.4 librosa soundfile pydub

In [3]:
# !pip install librosa

## 1. 라이브러리 및 데이터셋 로드

In [4]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import librosa

In [5]:
# 데이터셋 로드
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")

In [6]:
dataset[0]

{'audio': {'path': '0d38672e0bbdbdc460af55b8bb84a15b2730db2819f2af64f9c777d4d586f2de',
  'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00024414, 0.00048828,
         0.0005188 ]),
  'sampling_rate': 16000}}

### 1.1. 데이터셋 활용
- datasets / distil-whisper/librispeech_long
  - 허깅페이스에서 제공하는 음성 데이터셋으로, 주로 ASR평가를 위해 사용
  - 데이터 샘플
  
  ```python
  {
    "audio": {
        "path": "파일 경로 또는 파일 ID",
        "array": array([...]),  # PCM 데이터 배열
        "sampling_rate": 16000  # 샘플링 속도
    }
  }
  ```

In [7]:
sample = dataset[0]['audio']

### 1.2. 데이터셋 변환 (PCM -> pydub.AudioSegment)
- PCM(`Pulse Code Modulation`): 디지털 오디오 데이터의 기본 형식  

- **특징**
  - 1. 비압축 데이터
    - 압축 코덱(MP3, AAC)과 달리 압축되지 않은 원시의 데이터
  - 2. 샘플링
    - 아날로그 오디오 신호를 일정한 간격으로 측정하여 디지털 값으로 변환
  - 3. 샘플 폭
    - PCM이 차지하는 비트 수를 나타내는데, 16비트, 8비트 같은 게 있음.
  - 4. 채널
    - 오디오가 모노(1채널)인지, 스테레오(2채널)인지 결정.

- **장단점**
  - 장점
    - 1. 고품질 데이터
      - 압축되지 않은 원본 데이터를 제공하므로, 품질 손실이 없음
    - 2. 표준화
      - 오디오 처리 라이브러리나 딥러닝 모델에서 기본적으로 사용 가능
  - 단점
    - 1. 큰 파일 크기
      - 압축되지 않았기에 크기가 큼
    - 2. 직접 사용에 불편
      - 사람이 바로 이해하거나 사용 불가능한 형태로, 추가 처리가 필요함.
    

샘플 데이터를 짧게 보면

```python
{
    'audio':{
        'path': "파일 경로 또는 파일 ID",
        'array' : [0.0023, 0.0035, -0.0046, ...], #PCM 데이터 배열
        'sampling_rate' : 16000, # 샘플링 속도 (16,000Hz)
        'channels' : 1, # 모노(1채널)
    }
}
```

In [8]:
sample

{'path': '0d38672e0bbdbdc460af55b8bb84a15b2730db2819f2af64f9c777d4d586f2de',
 'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00024414, 0.00048828,
        0.0005188 ]),
 'sampling_rate': 16000}

In [9]:
import numpy as np
from pydub import AudioSegment

# 오디오 파일 로드

# 오디오 데이터 추출 및 AudioSegment 변환
audio_array = sample['array']
sampling_rate = sample['sampling_rate']

# PCM 데이터를 16비트 정수로 변환
audio_data = np.array(audio_array * 32767, dtype=np.int16)

# AudioSegment로 변환
audio = AudioSegment(
    audio_data.tobytes(),
    frame_rate=sampling_rate,
    sample_width=audio_data.dtype.itemsize,
    channels=1
)

# 오디오 분할 (30초 단위)
chunk_length_ms = 30 * 1000  # 30초
chunks = [audio[i:i + chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]

# 각 분할 저장
for i, chunk in enumerate(chunks):
    chunk.export(f"chunk_{i}.wav", format="wav")
    print(f"chunk_{i}.wav 저장 완료")

chunk_0.wav 저장 완료
chunk_1.wav 저장 완료
chunk_2.wav 저장 완료


In [10]:
chunks

## 2. 모델 불러오기 및 파이프라인 설정

In [11]:
import torch
# from transformers.models.whisper import EncoderDecoderCache

# CUDA 사용
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Whisper 모델 로드
model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, 
    torch_dtype=torch_dtype,
    # low_cpu_mem_usage=True, 
    use_safetensors=True
)
model.to(device)

# PCM 데이터를 Mel Spectorgram으로 변환 후 진행
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3-turbo") # PCM -> Mel Spectrogram 입력 변환을 위함

input_features = processor(
    audio_array,
    sampling_rate=sampling_rate,
    return_tensors="pt",
).input_features


# 신규 버전 업데이트) 기존 튜플을 EncoderDecoderCache로 변환
# past_key_values = EncoderDecoderCache.from_legacy_cache(past_key_values)
attention_mask = torch.ones_like(input_features)  # 모든 입력이 활성화된 상태로 설정

generates_ids = model.generate(
    input_features,
    temperature=0.7, # 다양성
    num_beams=2, # beam search를 사용한 텍스트 생성
    length_penalty=1.3, # 기본값 1, 긴 텍스트에 조금 더 유리하도록 설정
    attention_mask=attention_mask, # 입력 데이터에 대해 명시적으로 attention_mask 생성 후 전달
)
transcription = processor.batch_decode(generates_ids, skip_special_tokens=True)

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
c:\Users\dm705\AppData\Local\pypoetry\Cache\virtualenvs\ai_영어대화-UlUCYZce-py3.12\Lib\site-packages\transformers\generation\configuration_utils.py:695: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `1.3` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


In [12]:
print("Transcription:", transcription)

Transcription: [" Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind."]


### 3. 녹음한 파일로 ASR 테스트

In [13]:
import os
os.getcwd()

'c:\\Users\\dm705\\Study\\TIL\\Projects\\AI 영어대화'

In [14]:
file_name = 'self_motivation.m4a'
file_path = os.path.join(os.getcwd(), file_name)
print("파일 존재 여부:", os.path.exists(file_path))

파일 존재 여부: True


In [ ]:
# 이미 만들어진 processor, model을 사용

'''
model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, 
    torch_dtype=torch_dtype,
    # low_cpu_mem_usage=True, 
    use_safetensors=True
)
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3-turbo") # PCM -> Mel Spectrogram 입력 변환을 위함
'''

# Whisper모델은 PCM데이터 or WAV형식 데이터를 필요로 하기 때문에, 변환해줌.
import traceback # 전체 오류 스택 트레이스

# load file
try:
    record_file = AudioSegment.from_file(file_path, format="m4a")
    # convert to WAV format
    record_file.export("self_motivation.wav", format='wav')
    print("WAV 변환 성공")
except Exception as e:
    print("오디오 변환 실패:", e)
    traceback.print_exc()



WAV 변환 성공


FileNotFoundError가 발생하는 경우  
> traceback.print_exc() 를 사용해서, 전체 오류 스택 트레이스를 출력할 수 있다.    
> 나의 경우 `pydub`이 내부적으로 사용하는 `FFmpeg`가 설치되지 않아서 오류가 발생하는 것 같았음.  
> - `Couldn't find ffprobe or avprobe` warning이 발생했기 때문.  
> 때문에 `FFmpeg`를 아래와 같이 설치해줌.

---
#### 1. FFmpeg 다운로드
- [FFmpeg 공식 웹사이트 접속](https://ffmpeg.org/download.html)
- Windows 빌드 섹션에서 "Windows builds by gyan.dev" 링크 클릭.
#### 2. Windows 빌드 다운로드
- `ffmpeg-git-full.7z` 파일 다운로드
#### 3. 압축 해제 후 환경변수 설정
- 압축한 폴더의 `ffmpeg-bin` 폴더로 가서, 환경변수(시스템변수)로 추가.
- bash창을 다시 닫고 아래의 명령어 실행
- ```bash
    ffmpeg -version  
    ffprobe -version
    ```


In [16]:
# 재도전

# load file
try:
    record_file = AudioSegment.from_file(file_path, format="m4a")
    # convert to WAV format
    record_file.export("self_motivation.wav", format='wav')
    print("WAV 변환 성공")
except Exception as e:
    print("오디오 변환 실패:", e)
    traceback.print_exc()


WAV 변환 성공


In [17]:
# librosa를 이용한 파일 읽기
audio, sampling_rate = librosa.load(file_path, sr=16000)

print(f'오디오 데이터 길이: {len(audio)}')
print(f'샘플링 속도: {sampling_rate} Hz')



오디오 데이터 길이: 492203
샘플링 속도: 16000 Hz


C:\Users\dm705\AppData\Local\Temp\ipykernel_11336\1433415770.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sampling_rate = librosa.load(file_path, sr=16000)
c:\Users\dm705\AppData\Local\pypoetry\Cache\virtualenvs\ai_영어대화-UlUCYZce-py3.12\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


#### 3.1) 기본 파라미터로 진행

In [18]:
input_features = processor(
    audio,
    sampling_rate=16000,
    return_tensors='pt'
).input_features

generates_ids = model.generate(input_features)
transcription = processor.batch_decode(generates_ids, skip_special_tokens=True)

print("Transcription: ", transcription[0])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcription:   어떤 작업이든 지금부터 25분 동안 집중을 하고 5분 동안 휴식을 갖겠습니다. 앞으로 뽀모도로 기법을 활용해서 나의 하루를 바꿔나갈 것입니다. 나는 천재다. 나는 할 수 있다. 나는 좋은 일이 많이 생긴다. 나는 내가 원하는 모든 일을 이룰 수 있는 힘을 갖고 있다. 나는 준비가 되었고 나는 실행할 힘이 있다.


### 3.2) 하이퍼파라미터 튜닝 및 어텐션 마스크 제공

In [19]:
attention_mask = torch.ones_like(input_features)  # 모든 입력이 활성화된 상태로 설정

generates_ids = model.generate(
    input_features,
    temperature=0.7, # 다양성
    length_penalty=1.3, # 기본값 1, 긴 텍스트에 조금 더 유리하도록 설정
    attention_mask=attention_mask, # 입력 데이터에 대해 명시적으로 attention_mask 생성 후 전달
)
transcription = processor.batch_decode(generates_ids, skip_special_tokens=True)
print("Transcription: ", transcription[0])

c:\Users\dm705\AppData\Local\pypoetry\Cache\virtualenvs\ai_영어대화-UlUCYZce-py3.12\Lib\site-packages\transformers\generation\configuration_utils.py:695: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `1.3` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


Transcription:   어떤 작업이든 지금부터 25분 동안 집중을 하고 5분 동안 휴식을 갖겠습니다. 앞으로 뽀모도로 기법을 활용해서 나의 하루를 바꿔나갈 것입니다. 나는 천재다. 나는 할 수 있다. 나는 좋은 일이 많이 생긴다. 나는 내가 원하는 모든 일을 이룰 수 있는 힘을 갖고 있다. 나는 준비가 되었고 나는 실행할 힘이 있다.
